In [1]:
import os

import pandas as pd
from pandas import DataFrame
import numpy as np


from Constants import  Constants as const

In [2]:
sec_df = pd.read_stata(os.path.join(const.DATA_PATH, 'SECAnalytics.dta'))
sec_df['form'].unique()

array(['8-K', '8-K/A', '8-K12B', '8-K12G3', '8-K15D5', '18-K/A',
       '8-K12B/A', '8-K12G3/A'], dtype=object)

In [3]:
valid_8k_forms = ['8-K', '8-K/A', '8-K12B', '8-K12B/A', '8-K12G3', '8-K12G3/A', '8-K15D5']

df_8k = sec_df[sec_df['form'].isin(valid_8k_forms)].copy()

In [4]:
df_8k['rdate'] = pd.to_datetime(df_8k['rdate'])

In [5]:
df_8k[const.YEAR] = df_8k['rdate'].dt.year
df_8k[const.GVKEY] = pd.to_numeric(df_8k['gvkey'])

In [10]:
# === 构建变量 1：每年每家公司唯一 8-K filing 数量 ===
df_8k_filings = df_8k.drop_duplicates([const.GVKEY, const.YEAR, 'wrdsfname'], keep='first')
num_8k_filings = df_8k_filings.groupby([const.GVKEY, const.YEAR])['wrdsfname'].size().reset_index(name='num_8k_filings')

In [16]:
df_8k_filings['nitem'][0]

'2.01'

In [17]:
# === 构建变量 2：每年每家公司 8-K item 总数 ===
num_8k_items = df_8k_filings.groupby([const.GVKEY, const.YEAR])['nitemno'].sum(min_count=1).reset_index(name='num_8k_items')

# === 构建变量 3：Earnings-related item 数量 ===
earnings_items = {'2.02', '7.01', '8.01', '1.01', '1.02', '2.01', '2.04','2.05', '2.06'}
df_8k['is_earn'] = df_8k['nitem'].isin(earnings_items)
num_earn_items = df_8k[df_8k['is_earn']].groupby([const.GVKEY, const.YEAR]).size().reset_index(name='num_earn_items')


In [21]:
# === 合并所有结果 ===
df_summary = num_8k_filings \
    .merge(num_8k_items, on=[const.GVKEY, const.YEAR], how='outer') \
    .merge(num_earn_items, on=[const.GVKEY, const.YEAR], how='outer')
df_summary['num_earn_items'] = df_summary['num_earn_items'].fillna(0)
df_summary['num_nonearn_items'] = df_summary['num_8k_items'] - df_summary['num_earn_items']

In [23]:
df_summary.keys()

Index(['gvkey', 'fiscal_year', 'num_8k_filings', 'num_8k_items',
       'num_earn_items', 'num_nonearn_items'],
      dtype='object')

In [22]:
reg_df = pd.read_stata(os.path.join(const.RESULT_PATH, '20250604_stock_act_reg_data_v1.dta'))

reg_df2 = reg_df.merge(df_summary, on=[const.GVKEY, const.YEAR], how='left')
reg_df2.describe()

,gvkey,fiscal_year,mf_indicator,frequency,log_frequency,MajorGovCustomer,post,log_market_value,lev,ROA,...,RegIn_Time_my_high,RegIn_Dollar_my_avg,RegIn_Dollar_my_high,latitude,longitude,distance_to_dc_km,num_8k_filings,num_8k_items,num_earn_items,num_nonearn_items
count,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,...,12074.000000,8786.000000,12074.000000,12074.000000,12074.000000,12074.000000,10545.000000,10545.000000,10545.000000,10545.000000
mean,81220.205317,2011.546960,0.363508,4.817459,0.868412,0.153553,0.508945,6.517410,0.181687,-0.054435,...,0.500083,96.426826,0.500083,37.982031,-87.575054,1779.385976,13.118160,26.813940,11.951257,14.862684
min,1004.000000,2008.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.786519,0.000000,-33.150838,...,0.000000,14.131963,0.000000,-34.605469,-157.883326,1.101796,1.000000,1.000000,0.000000,0.000000
25%,12142.000000,2010.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.012064,0.000930,-0.039075,...,0.000000,91.059692,0.000000,33.975391,-98.393418,493.131210,8.000000,16.000000,6.000000,9.000000
50%,62592.000000,2012.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.520895,0.148584,0.027111,...,1.000000,94.937225,1.000000,39.287801,-86.137865,1195.185900,11.000000,23.000000,10.000000,13.000000
75%,156153.000000,2014.000000,1.000000,7.000000,2.079442,0.000000,1.000000,7.976261,0.264176,0.066365,...,1.000000,101.609390,1.000000,41.673379,-75.550073,2419.348096,16.000000,33.000000,15.000000,18.000000
max,296753.000000,2015.000000,1.000000,36.000000,3.610918,1.000000,1.000000,11.561005,17.825450,1.247209,...,1.000000,139.373062,1.000000,61.186678,152.963505,15710.188305,162.000000,321.000000,157.000000,164.000000
std,73644.923742,2.279738,0.481029,8.551352,1.245300,0.360535,0.499941,2.163776,0.273871,0.496132,...,0.500021,10.007222,0.500021,6.698589,29.216612,1781.528276,7.763607,16.834492,8.706231,8.848418


In [24]:
for key in ['num_8k_filings', 'num_8k_items', 'num_earn_items', 'num_nonearn_items']:
    reg_df2[key] = reg_df2[key].fillna(0)
    reg_df2['ln_' + key] = reg_df2[key].apply(lambda x: np.log(x + 1))

reg_df2.to_stata(os.path.join(const.RESULT_PATH, '20250704_stock_act_reg_data_v1.dta'), write_index=False, version=119)